## Import libraries

In [212]:
import pandas as pd
import requests
import tabula

## Import Consumer Expenditure Surveys, U.S. Bureau of Labor Statistics, 2019-2020 (Northeast US)

In [213]:
#using tabula to parse pdf
dfs = tabula.read_pdf("https://www.bls.gov/cex/tables/cross-tab/mean/cu-region-by-income-northeast-2020.pdf", pages = "all")

## Find Income + Total Expenditures of Income Groups
- Low-income group is population of households that earns between $15,000 - $30,000.
- High-income group is population of households that earns between $100,000 - $149,999.

In [226]:
#select columns of income groups under analysis
dfSES = dfs[0][['Unnamed: 0','$15,000','$100,000']]
dfSES.columns = ['category', "lowIncomeExpense","highIncomeExpense"]

dfIncomeExpense = dfSES.loc[[9,49]]
dfIncomeExpense['lowIncomeExpense'] = dfIncomeExpense['lowIncomeExpense'].str.replace(",","").str.replace("$","",regex=False).astype(float)
dfIncomeExpense['highIncomeExpense'] = dfIncomeExpense['highIncomeExpense'].str.replace(",","").str.replace("$","",regex=False).astype(float)

dfIncomeExpense = dfIncomeExpense.T
dfIncomeExpense.columns = ['income','expenses']
dfIncomeExpense = dfIncomeExpense[1:]
dfIncomeExpense.index = ["lowIncome","highIncome"]
dfIncomeExpense


,income,expenses
lowIncome,22498.0,34970.0
highIncome,121558.0,85111.0


## Find Expenditure Categories From the Consumer Expenditure Surveys
- Pick out major expenditure categories that match the BLS' Consumer Price Index estimates and calculate how much each income group spent on the various categories. This is required in order to estimate their inflationary cost in dollar amounts.

In [215]:
dfExpense = pd.concat(dfs).reset_index(drop=True)[['Unnamed: 0','$15,000','$100,000']]
dfExpense.columns = ['category', "lowIncomeExpense","highIncomeExpense"]

#These rows correspond to major category groups that match up to BLS' inflation categories.
dfExpenseToMatch = dfExpense.loc[[51,52,83,85,88,96,105,108,112,120,131,145,151,169,165,167,171,173,175,177]]
dfExpenseToMatch['lowIncomeExpense'] = dfExpenseToMatch['lowIncomeExpense'].str.replace(",","").str.replace("$","",regex=False).astype(float)
dfExpenseToMatch['highIncomeExpense'] = dfExpenseToMatch['highIncomeExpense'].str.replace(",","").str.replace("$","",regex=False).astype(float)
dfExpenseToMatch['category'] = dfExpenseToMatch['category'].str.replace(".","",regex=False).str.rstrip()
dfExpenseToMatch = dfExpenseToMatch.reset_index(drop=True)

#rename and group categories
dfExpenseToMatch = dfExpenseToMatch.replace("Household operations","Household furnishings and operations",regex=True) \
                                   .replace("Housekeeping supplies","Household furnishings and operations",regex=True) \
                                   .replace("Household furnishings and equipment","Household furnishings and operations",regex=True) \
                                   .replace("Reading","Other goods and services",regex=True) \
                                   .replace("Tobacco products and smoking supplies","Other goods and services",regex=True) \
                                   .replace("Miscellaneous","Other goods and services",regex=True) \
                                   .replace("Cash contributions","Other goods and services",regex=True) \
                                   .replace("Personal insurance and pensions","Other goods and services",regex=True)

#hack for string issue with one expenditure category?
dfExpenseToMatch = dfExpenseToMatch.replace("Personal care products and services",dfExpenseToMatch['category'][13],regex=True)
dfExpenseToMatch = dfExpenseToMatch.groupby("category").sum().reset_index()


In [140]:
dfExpenseToMatch

,category,lowIncomeExpense,highIncomeExpense
0,Alcoholic beverages,212.0,938.0
1,Apparel and services,996.0,3077.0
2,Education,858.0,3472.0
3,Entertainment,1397.0,3854.0
4,Food,4667.0,10904.0
5,Food at home,3323.0,6759.0
6,Food away from home,1345.0,4145.0
7,Healthcare,3653.0,6318.0
8,Household furnishings and operations,2326.0,5459.0
9,Other goods and services,2934.0,15584.0


## Import Consumer Price Index, U.S. Bureau of Labor Statistics, Northeast Region – May 2022
- Get inflation rate figures from monethly BLS CPI estimates for various categories and match them to the categories in the BLS Consumer Expenditure Survey.

In [141]:
tablesInflation = pd.read_html('https://www.bls.gov/regions/mid-atlantic/news-release/consumerpriceindex_northeast.htm')
dfInflation = tablesInflation[3]
dfInflation = dfInflation['Expenditure category']['Expenditure category'].to_frame().join(dfInflation['Percent change from']['Apr.2021'])

In [217]:
#pick out categories to match CES
dfInflationToMatch = dfInflation.loc[[4,5,12,13,16,20,25,27,29,43,48,50,53]]
dfInflationToMatch = dfInflationToMatch.reset_index(drop=True)

dfInflationToMatch.columns = ['category','inflationRate']

#rename rows
dfInflationToMatch['category'].loc[5] = "Utilities, fuels, and public services"
dfInflationToMatch['category'].loc[7] = "Apparel and services"
dfInflationToMatch['category'].loc[9] = "Healthcare"
dfInflationToMatch['category'].loc[10] = "Entertainment"
dfInflationToMatch['category'].loc[11] = "Education"
dfInflationToMatch['inflationRate'] = dfInflationToMatch['inflationRate'].astype(float)
dfInflationToMatch

,category,inflationRate
0,Food,8.0
1,Food at home,9.3
2,Food away from home,5.8
3,Alcoholic beverages,4.1
4,Shelter,3.1
5,"Utilities, fuels, and public services",21.3
6,Household furnishings and operations,9.3
7,Apparel and services,3.0
8,Transportation,18.2
9,Healthcare,3.3


## Merge Inflation + Expense Data, Estimate Total Inflation Cost
- Merge inflation and expense datasets, calculate additional expenditure due to inflation for each category.

In [158]:
dfInflationExpense = pd.merge(dfInflationToMatch,dfExpenseToMatch,on="category")

In [216]:
dfInflationExpense['lowIncomeInflation'] = dfInflationExpense['lowIncomeExpense'] *dfInflationExpense['inflationRate'] / 100
dfInflationExpense['highIncomeInflation'] = dfInflationExpense['highIncomeExpense'] *dfInflationExpense['inflationRate'] / 100
dfInflationExpense

,category,inflationRate,lowIncomeExpense,highIncomeExpense,lowIncomeInflation,highIncomeInflation
0,Food,8.0,4667.0,10904.0,373.360,872.320
1,Food at home,9.3,3323.0,6759.0,309.039,628.587
2,Food away from home,5.8,1345.0,4145.0,78.010,240.410
3,Alcoholic beverages,4.1,212.0,938.0,8.692,38.458
4,Shelter,3.1,10963.0,16537.0,339.853,512.647
5,"Utilities, fuels, and public services",21.3,2906.0,5264.0,618.978,1121.232
6,Household furnishings and operations,9.3,2326.0,5459.0,216.318,507.687
7,Apparel and services,3.0,996.0,3077.0,29.880,92.310
8,Transportation,18.2,4058.0,13705.0,738.556,2494.310
9,Healthcare,3.3,3653.0,6318.0,120.549,208.494


In [229]:
#exclude first row (Food) since next two rows are components of that.
dfIncomeExpense['inflation'] = [dfInflationExpense['lowIncomeInflation'][1:].sum().round(),dfInflationExpense['highIncomeInflation'][1:].sum().round()]
dfIncomeExpenseInflation = dfIncomeExpense

In [230]:
dfIncomeExpenseInflation

,income,expenses,inflation
lowIncome,22498.0,34970.0,2713.0
highIncome,121558.0,85111.0,6991.0


## Calculate Inflation as Portion of Income
- Take the additional inflation expenditures of each category as a proportion of an income group's average income to find the percentage of an income's groups income that has been taken up by inflationary pressure for certain categories.

In [227]:
#pick out relevant rows
dfInflationIncome = dfInflationExpense.loc[[0,8,4,5,9]]

#take proportion and round
dfInflationIncome['lowIncomeInflationProp'] = dfInflationIncome['lowIncomeInflation'] / dfIncomeExpense['income']['lowIncome'] * 100
dfInflationIncome['highIncomeInflationProp'] = dfInflationIncome['highIncomeInflation'] / dfIncomeExpense['income']['highIncome'] * 100
dfInflationIncome['lowIncomeInflationProp'] = dfInflationIncome['lowIncomeInflationProp'].round(1)
dfInflationIncome['highIncomeInflationProp'] = dfInflationIncome['highIncomeInflationProp'].round(1)

dfInflationIncome = dfInflationIncome[['category','inflationRate','lowIncomeInflationProp','highIncomeInflationProp']]
dfInflationIncome



,category,inflationRate,lowIncomeInflationProp,highIncomeInflationProp
0,Food,8.0,1.7,0.7
8,Transportation,18.2,3.3,2.1
4,Shelter,3.1,1.5,0.4
5,"Utilities, fuels, and public services",21.3,2.8,0.9
9,Healthcare,3.3,0.5,0.2


## Get Other Inflation Rates We Care About
- These are other inflation rates that are refernced in the story.

In [222]:
dfOtherRates = dfInflation.loc[[0,15,4,20,37,23,24]]
dfOtherRates = dfOtherRates.reset_index(drop=True)
dfOtherRates.columns = ['category','inflationRate']

In [223]:
dfOtherRates

,category,inflationRate
0,All items,7.2
1,Housing,5.6
2,Food,8.0
3,Fuels and utilities,21.3
4,Gasoline (all types),43.6
5,Electricity,14.9
6,Utility (piped) gas service,23.7
